<a href="https://colab.research.google.com/github/suryansh29/DL-project/blob/main/few_shot_learning_kl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers
!pip install datasets
!pip install trl
!pip install evaluate



  Using cached trl-0.8.6-py3-none-any.whl (245 kB)
  Using cached accelerate-0.29.3-py3-none-any.whl (297 kB)
  Using cached tyro-0.8.3-py3-none-any.whl (102 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylin

In [2]:
from huggingface_hub import notebook_login

notebook_login()

In [24]:
from datasets import load_dataset, ClassLabel, concatenate_datasets

mnli = load_dataset("nyu-mll/multi_nli")

In [25]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer

tokenizer = AutoTokenizer.from_pretrained("facebook/opt-350m")
# model = AutoModelForSequenceClassification.from_pretrained("facebook/opt-350m", num_labels=len(tokenizer))
model = AutoModelForSequenceClassification.from_pretrained("facebook/opt-350m", num_labels=2)

Some weights of OPTForSequenceClassification were not initialized from the model checkpoint at facebook/opt-350m and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [26]:
PATTERN="Premise: {text1}, Hypothesis: {text2}. Does the hypothesis follow the premise?"
TARGET_PREFIX=" "

In [27]:
# target_tokens_ids = tokenizer.convert_tokens_to_ids(['ĠYes', 'ĠNo'])
target_tokens_ids = [0,1]


In [3]:
target_tokens_ids

[0, 1]

In [28]:
from datasets import ClassLabel

mnli = mnli.filter(lambda example: example["label"] != 1 and example["label"] != -1)

# change labels of contradiction examples from 2 to 1
def change_label(example):
    example["label"] = 1 if example["label"] == 2 else example["label"]
    return example
mnli = mnli.map(change_label)





In [29]:
mnli
 # change features to reflect the new labels
features = mnli["train"].features.copy()
features["label"] = ClassLabel(num_classes=2)

# features["label"] = ClassLabel(num_classes=len(tokenizer))
mnli = mnli.cast(features)  # overwrite old features

Casting the dataset:   0%|          | 0/261802 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/6692 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/6703 [00:00<?, ? examples/s]

In [30]:
def preprocess_function_without_context(examples):
  pattern_examples = [
          PATTERN.format(
              text1=examples['premise'][idx],
              text2=examples['hypothesis'][idx]) + TARGET_PREFIX
          for idx in range(len(examples['premise']))
      ]

  args = (pattern_examples,)
  result = tokenizer(*args, padding="max_length", truncation=True)

# Get tokens
  result["input_tokens"] = [tokenizer.convert_ids_to_tokens(
      ids) for ids in result["input_ids"]]

  # Decode input
  result["input_text"] = [tokenizer.decode(
      ids) for ids in result["input_ids"]]

  result["label"] = [target_tokens_ids[l] for l in examples["label"]]

  return result


In [31]:
tokenized_mnli_without_context = mnli.map(preprocess_function_without_context, batched=True)

Map:   0%|          | 0/261802 [00:00<?, ? examples/s]

Asking to pad to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no padding.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Map:   0%|          | 0/6692 [00:00<?, ? examples/s]

Map:   0%|          | 0/6703 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['promptID', 'pairID', 'premise', 'premise_binary_parse', 'premise_parse', 'hypothesis', 'hypothesis_binary_parse', 'hypothesis_parse', 'genre', 'label', 'input_ids', 'attention_mask', 'input_tokens', 'input_text'],
        num_rows: 261802
    })
    validation_matched: Dataset({
        features: ['promptID', 'pairID', 'premise', 'premise_binary_parse', 'premise_parse', 'hypothesis', 'hypothesis_binary_parse', 'hypothesis_parse', 'genre', 'label', 'input_ids', 'attention_mask', 'input_tokens', 'input_text'],
        num_rows: 6692
    })
    validation_mismatched: Dataset({
        features: ['promptID', 'pairID', 'premise', 'premise_binary_parse', 'premise_parse', 'hypothesis', 'hypothesis_binary_parse', 'hypothesis_parse', 'genre', 'label', 'input_ids', 'attention_mask', 'input_tokens', 'input_text'],
        num_rows: 6703
    })
})

In [34]:
import numpy as np

def create_few_shot_examples(examples, num_shots=3):
  indices = np.random.choice(range(len(examples['premise'])), size=num_shots, replace=False)
  separate_shots_by = "\n\n"
  context=""
  for idx in indices:
    formated_sample = PATTERN.format(
        text1=examples['premise'][idx],
        text2=examples['hypothesis'][idx]
    )

    verbalized_label = "ĠYes" if examples['label'][idx] == 0 else "ĠNo"
    # verbalized_label = tokenizer.convert_ids_to_tokens([(examples['label'][idx])])[0]
    if verbalized_label.startswith("Ġ"):
      verbalized_label = verbalized_label[1:]
    context += f"{formated_sample}{TARGET_PREFIX}{verbalized_label}{separate_shots_by}"
  return context



def preprocess_function_with_few_shot_context(examples):
  examples["label"] = [target_tokens_ids[l] for l in examples["label"]]
  pattern_examples = [
          create_few_shot_examples(examples, 3) +
          PATTERN.format(
              text1=examples['premise'][idx],
              text2=examples['hypothesis'][idx]) + TARGET_PREFIX
          for idx in range(len(examples['premise']))
      ]
  args = (pattern_examples,)
  result = tokenizer(*args, padding="max_length", truncation=True)

# Get tokens
  result["input_tokens"] = [tokenizer.convert_ids_to_tokens(
      ids) for ids in result["input_ids"]]

  # Decode input
  result["input_text"] = [tokenizer.decode(
      ids) for ids in result["input_ids"]]

  result["label"] = examples["label"]

  return result


In [35]:
tokenized_mnli_validation_matched_with_context = mnli['validation_matched'].map(preprocess_function_with_few_shot_context, batched=True)
tokenized_mnli_validation_mismatched_with_context = mnli['validation_mismatched'].map(preprocess_function_with_few_shot_context, batched=True)
# tokenized_mnli_training_with_context = mnli['train'].map(preprocess_function_with_few_shot_context, batched=True)

Map:   0%|          | 0/6692 [00:00<?, ? examples/s]

Map:   0%|          | 0/6703 [00:00<?, ? examples/s]

In [37]:
tokenized_mnli_validation_matched_with_context['input_text']

['</s>Premise: In the moment of victory, Tuppence betrayed a somewhat unsportsmanlike triumph., Hypothesis: Tuppence was triumphant.. Does the hypothesis follow the premise? Yes\n\nPremise: Czesiek had suitable experience in the matter., Hypothesis: The Czesiek had plenty of experience.. Does the hypothesis follow the premise? Yes\n\nPremise: There\'s only one thing for me to do.", Hypothesis: There\'s a few things left for me to do.. Does the hypothesis follow the premise? No\n\nPremise: This site includes a list of all award winners and a searchable database of Government Executive articles., Hypothesis: The Government Executive articles housed on the website are not able to be searched.. Does the hypothesis follow the premise? ',
 "</s>Premise: Another quarter billion plus dollars of the total amount sought was earmarked to pay down operating debt accrued in past years., Hypothesis: Another quarter billion dollars or more was earmarked to pay down debt.. Does the hypothesis follow t

In [38]:
import evaluate

accuracy = evaluate.load("accuracy")

global_eval_pred = None

def compute_metrics(eval_pred):
  global global_eval_pred
  global_eval_pred = eval_pred
  predictions, labels = eval_pred
  predictions = np.argmax(predictions, axis=1)
  print('predictions, labels: ', predictions, labels)
  return accuracy.compute(predictions=predictions, references=labels)

In [39]:
hans = load_dataset("hans")


In [40]:
 # change features to reflect the new labels
features = hans["train"].features.copy()
features["label"] = ClassLabel(num_classes=2)

# features["label"] = ClassLabel(num_classes=len(tokenizer))
hans = hans.cast(features)  # overwrite old features

Casting the dataset:   0%|          | 0/30000 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/30000 [00:00<?, ? examples/s]

In [41]:
tokenized_hans_without_context = hans.map(preprocess_function_without_context, batched=True)

Map:   0%|          | 0/30000 [00:00<?, ? examples/s]

Map:   0%|          | 0/30000 [00:00<?, ? examples/s]

In [42]:
from datasets import concatenate_datasets

# Get validation_matched and validation_mismatched datasets
validation_matched_dataset_without_context = tokenized_mnli_without_context["validation_matched"]
validation_mismatched_dataset_without_context = tokenized_mnli_without_context["validation_mismatched"]

# Concatenate the two datasets
combined_validation_dataset_without_context = concatenate_datasets([validation_matched_dataset_without_context, validation_mismatched_dataset_without_context])

# Now combined_validation_dataset contains both validation_matched and validation_mismatched datasets

# Optionally, shuffle the combined dataset
combined_validation_dataset_without_context = combined_validation_dataset_without_context.shuffle()


# Concatenate the two datasets
combined_validation_dataset_with_context = concatenate_datasets([tokenized_mnli_validation_matched_with_context, tokenized_mnli_validation_mismatched_with_context])

# Optionally, shuffle the combined dataset
combined_validation_dataset_with_context = combined_validation_dataset_with_context.shuffle()

**Zero Shot Inference In-Domain **

In [43]:
training_args = TrainingArguments(
    output_dir="zero_shot_mnli_validation",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    # train_dataset=None,
    eval_dataset=combined_validation_dataset_without_context,
    tokenizer=tokenizer,
    # data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.evaluate()

predictions, labels:  [0 0 0 ... 0 0 0] [0 0 1 ... 0 1 0]


{'eval_loss': 1.1591137647628784,
 'eval_accuracy': 0.5127286300858529,
 'eval_runtime': 249.2898,
 'eval_samples_per_second': 53.733,
 'eval_steps_per_second': 3.362}

In [ ]:
tokenized_mnli_validation_matched_with_context['input_tokens']

In [25]:
tokenizer.convert_ids_to_text([440])

AttributeError: 'GPT2TokenizerFast' object has no attribute 'convert_ids_to_text'

**Zero Shot Inference Out-Domain **

In [ ]:
# training_args = TrainingArguments(
#     output_dir="zero_shot_mnli_validation",
#     learning_rate=2e-5,
#     per_device_train_batch_size=16,
#     per_device_eval_batch_size=16,
#     num_train_epochs=2,
#     weight_decay=0.01,
#     evaluation_strategy="epoch",
#     save_strategy="epoch",
#     load_best_model_at_end=True,
#     push_to_hub=True,
# )

trainer = Trainer(
    model=model,
    # args=training_args,
    eval_dataset=hans['validation'],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

trainer.evaluate()

Few shot inference (ICL) out of domain
*italicised text*

In [44]:
tokenized_hans_validation_with_context = hans['validation'].map(preprocess_function_with_few_shot_context, batched=True)



Map:   0%|          | 0/30000 [00:00<?, ? examples/s]

In [22]:
training_args = TrainingArguments(
    output_dir="few_shot_hans_validation",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=True,
)

trainer = Trainer(
    args=training_args,

    model=model,
    eval_dataset=tokenized_hans_validation_with_context,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

trainer.evaluate()

predictions, labels:  [0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0] [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]


{'eval_loss': 0.8382080793380737,
 'eval_accuracy': 0.06,
 'eval_runtime': 2.5344,
 'eval_samples_per_second': 39.456,
 'eval_steps_per_second': 2.762}

In [56]:
copy_global_eval_pred = global_eval_pred

In [59]:
import evaluate

accuracy = evaluate.load("accuracy")

predictions, labels = copy_global_eval_pred
# predictions = np.argmax(predictions, axis=1)
# print('predictions, labels: ', predictions, labels)
# accuracy.compute(predictions=predictions, references=labels)

In [73]:
model(tokenized_hans_validation_with_context.select(np.arange(1))['input_ids'])

AttributeError: 'list' object has no attribute 'size'

In [65]:
np.argmax(predictions, axis=1)

array([24652, 30450, 30450,  2291,  2291, 30450, 30450, 32172, 30450,
       30194, 24652, 30450, 30450, 24652, 30450, 30450, 30194, 30450,
       30450, 30450, 30450, 32172, 30450, 32172, 24652, 30450, 30450,
       24652, 30450, 30450, 30450, 30450, 30450, 30450, 30450, 30450,
        2291, 30450, 30450, 30450, 30450, 30450, 30450, 32172,  2291,
       30450, 30450,  2291, 24652, 32172, 24652, 30450, 32172, 30450,
       30450, 30194, 30450, 30450, 32172, 30194, 30450, 30450, 30450,
       30450,  2291, 32172, 24652, 30450, 19742,  2291, 30450, 30194,
       30450, 30450, 32172, 30450, 30450, 30450,  2291, 24652, 30450,
       32172, 30450, 24652,  2291, 32172, 32172, 30450, 30450, 30450,
       30450, 30450, 30194, 24652,  2291, 30450, 30450, 30450, 30450,
       30450])

In [38]:
tokenized_hans_validation_with_context.select(np.arange(1))['input_tokens']

[['</s>',
  'Prem',
  'ise',
  ':',
  'ĠThe',
  'Ġtourist',
  'Ġcontacted',
  'Ġthe',
  'Ġsecretaries',
  'Ġ.',
  ',',
  'ĠHyp',
  'ot',
  'hesis',
  ':',
  'ĠThe',
  'Ġsecretaries',
  'Ġcontacted',
  'Ġthe',
  'Ġtourist',
  'Ġ..',
  'ĠDoes',
  'Ġthe',
  'Ġhypothesis',
  'Ġfollow',
  'Ġthe',
  'Ġpremise',
  '?',
  'ĠNo',
  'Ċ',
  'Ċ',
  'Prem',
  'ise',
  ':',
  'ĠThe',
  'Ġscientists',
  'Ġrecommended',
  'Ġthe',
  'Ġathlete',
  'Ġ.',
  ',',
  'ĠHyp',
  'ot',
  'hesis',
  ':',
  'ĠThe',
  'Ġathlete',
  'Ġrecommended',
  'Ġthe',
  'Ġscientists',
  'Ġ..',
  'ĠDoes',
  'Ġthe',
  'Ġhypothesis',
  'Ġfollow',
  'Ġthe',
  'Ġpremise',
  '?',
  'ĠNo',
  'Ċ',
  'Ċ',
  'Prem',
  'ise',
  ':',
  'ĠThe',
  'Ġdoctor',
  'Ġadmired',
  'Ġthe',
  'Ġauthors',
  'Ġ.',
  ',',
  'ĠHyp',
  'ot',
  'hesis',
  ':',
  'ĠThe',
  'Ġauthors',
  'Ġadmired',
  'Ġthe',
  'Ġdoctor',
  'Ġ..',
  'ĠDoes',
  'Ġthe',
  'Ġhypothesis',
  'Ġfollow',
  'Ġthe',
  'Ġpremise',
  '?',
  'ĠNo',
  'Ċ',
  'Ċ',
  'Prem',
  'ise',
  

In [39]:
tokenized_hans_validation_with_context.select(np.arange(1))['input_text']

['</s>Premise: The tourist contacted the secretaries., Hypothesis: The secretaries contacted the tourist.. Does the hypothesis follow the premise? No\n\nPremise: The scientists recommended the athlete., Hypothesis: The athlete recommended the scientists.. Does the hypothesis follow the premise? No\n\nPremise: The doctor admired the authors., Hypothesis: The authors admired the doctor.. Does the hypothesis follow the premise? No\n\nPremise: The president advised the doctor., Hypothesis: The doctor advised the president.. Does the hypothesis follow the premise? ']

In [51]:
tokenizer.convert_ids_to_tokens([6454])

['IO']

Few shot inference (ICL) in domain


In [45]:
trainer = Trainer(
    model=model,
    eval_dataset=combined_validation_dataset_with_context,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

trainer.evaluate()

predictions, labels:  [1 1 1 ... 1 1 0] [0 0 1 ... 0 1 0]


{'eval_loss': 0.928181529045105,
 'eval_accuracy': 0.48659947741694665,
 'eval_runtime': 779.0324,
 'eval_samples_per_second': 17.194,
 'eval_steps_per_second': 2.15}

Finetuning

In [107]:
model_to_ft = AutoModelForSequenceClassification.from_pretrained("facebook/opt-350m", num_labels=len(tokenizer))

KL divergence based loss

In [ ]:


training_args = TrainingArguments(
    output_dir="context_distillation",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=True,
)

trainer = Trainer(
    model=model_to_ft,
    args=training_args,
    train_dataset=None,
    eval_dataset=combined_validation_dataset_without_context,
    tokenizer=tokenizer,
    # data_collator=data_collator,
    compute_metrics=compute_metrics,
)


For each, calculate in-domain accuracy (on mnli) and out-domain accuracy (on hans)

1) Try out acuracy for zero shot inference without fine-tuning
2) Try out accuracy for few shot inference (in-context learning) without fine-tuning
3) Try out LoRA, QLoRA and PEFT based fine-tuning
4) KL divergence loss based fine-tuning using LoRA/QLoRa, PEFT